# Gemma 한국어 요약 모델 파인튜닝 빠르게 해보기
- https://devocean.sk.com/blog/techBoardDetail.do?ID=165703

In [1]:
# !pip3 install -q -U transformers==4.38.2
# !pip3 install -q -U datasets==2.18.0
# !pip3 install -q -U bitsandbytes==0.42.0
# !pip3 install -q -U peft==0.9.0
# !pip3 install -q -U trl==0.7.11
# !pip3 install -q -U accelerate==0.27.2

In [2]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")

In [5]:
# dataset
# dataset['train'][0]

In [6]:
# BASE_MODEL = "google/gemma-2b-it"

# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

In [7]:
doc = dataset['train']['document'][0]

In [8]:
# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [9]:
# messages = [
#     {
#         "role": "user",
#         "content": "다음 글을 요약해주세요:\n\n{}".format(doc)
#     }
# ]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [10]:
# prompt

In [11]:
# outputs = pipe(
#     prompt,
#     do_sample=True,
#     temperature=0.2,
#     top_k=50,
#     top_p=0.95,
#     add_special_tokens=True
# )

In [12]:
# print(outputs[0]["generated_text"][len(prompt):])

# 파인 튜닝

In [13]:
torch.cuda.empty_cache()

In [14]:
def generate_prompts(example):
    output_texts = []
    for i in range(len(example['document'])):
        messages = [
            {"role": "user",
             "content": "다음 글을 요약해주세요:\n\n {}".format(example['document'][i])}, 
            {"role": "assistant",
             "content": "{}".format(example['summary'][i])}
        ]
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(chat_message)

    return output_texts

In [15]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['document'])):
        prompt_list.append(r"""<bos><start_of_turn>user
다음 글을 요약해주세요:

{}<end_of_turn>
<start_of_turn>model
{}<end_of_turn><eos>""".format(example['document'][i], example['summary'][i]))
    return prompt_list

In [16]:
train_data = dataset['train']
# print(generate_prompt(train_data[:1])[0])

In [17]:
lora_config = LoraConfig(
    r=6,
    lora_alpha = 8,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16
# )

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [18]:
BASE_MODEL = "google/gemma-2b-it"

# device_map={'': torch.cuda.current_device()}
# model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map=device_map, quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, 
    torch_dtype=torch.float16,
    device_map="auto", 
    quantization_config=bnb_config
    )


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)

lora_r  = 8
lora_alpha = 32
lora_dropout = 0.05
# target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],

config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    # target_modules=target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, config)

In [23]:
import transformers

train_args = transformers.TrainingArguments(
        output_dir="outputs",
#        num_train_epochs = 1,
        max_steps=3000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),

In [28]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=train_args,
    # peft_config=lora_config,
    formatting_func=generate_prompt,
)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


AttributeError: 'tuple' object has no attribute 'remove_unused_columns'

In [26]:
# trainer = transformers.Trainer(
#     model=model,
#     train_dataset=train_data,
#     args=train_args,
#     formatting_func=generate_prompt,
#     ),


TypeError: Trainer.__init__() got an unexpected keyword argument 'formatting_func'

In [ ]:
trainer.train()

In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [25]:
trainer.train()

ValueError: You can't train a model that has been loaded in 8-bit precision on a different device than the one you're training on. Make sure you loaded the model on the correct device using for example `device_map={'':torch.cuda.current_device() or device_map={'':torch.xpu.current_device()}